In [ ]:
from google.colab import drive
 
drive.mount('/content/gdrive')

# **Import Library**

In [ ]:
import pandas as pd
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn import metrics
import matplotlib.pyplot as plt

# **Data Exploration**

**1. Data Training**

In [ ]:
df = pd.read_excel('/content/gdrive/My Drive/BDC SD/Salinan Data Latih BDC.xlsx')

In [ ]:
df.head(10)

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,2020-08-17 00:00:00,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,2020-07-17 00:00:00,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,2020-07-13 00:00:00,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,2020-07-08 00:00:00,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,2020-06-24 00:00:00,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg
5,736,1,2020-06-17 00:00:00,event promo smartphone JNE 2020 spesial di bul...,"selamat siang teman teman fb ku semuanya🤩,cuma...",736.png
6,1161,1,2020-05-05 00:00:00,MEREKA SUDAH MEMPERSIAPKAN DIRI DENGAN BAIK UN...,LIHATLAH MEREKA SUDAH MEMPERSIAPKAN DIRI DENGA...,1161.png
7,1597,1,2020-03-24 00:00:00,Foto Presiden Italia menangis karena tak cukup...,Italia punya fasilitas perawatan kesehatan ter...,1597.png
8,2098,1,2020-01-09 00:00:00,"Kapolres Timor Tengah Utara , Nusa Tenggara Ti...",Polisi goblok. Bukanya cpet” d tolongin malah ...,2098.jpg
9,2226,1,2019-12-24 00:00:00,Video Polisi china telah menganiaya wanita uig...,Polisi china telah menganiaya wanita uighur le...,2226.png


In [ ]:
df.shape

(4231, 6)

In [ ]:
df = df.drop(['tanggal', 'nama file gambar'], axis=1)

In [ ]:
df = df.set_index('ID')

In [ ]:
df.head(10)

,label,judul,narasi
ID,,,
71,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...
461,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...
495,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...
550,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k..."
681,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .
736,1,event promo smartphone JNE 2020 spesial di bul...,"selamat siang teman teman fb ku semuanya🤩,cuma..."
1161,1,MEREKA SUDAH MEMPERSIAPKAN DIRI DENGAN BAIK UN...,LIHATLAH MEREKA SUDAH MEMPERSIAPKAN DIRI DENGA...
1597,1,Foto Presiden Italia menangis karena tak cukup...,Italia punya fasilitas perawatan kesehatan ter...
2098,1,"Kapolres Timor Tengah Utara , Nusa Tenggara Ti...",Polisi goblok. Bukanya cpet” d tolongin malah ...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4231 entries, 71 to 999860
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   4231 non-null   int64 
 1   judul   4231 non-null   object
 2   narasi  4231 non-null   object
dtypes: int64(1), object(2)
memory usage: 132.2+ KB


In [ ]:
df['label'].value_counts()

1    3465
0     766
Name: label, dtype: int64

Dari label, terlihat data unbalanced karena label hoax(1) jauh lebih banyak daripada label 0.

Menggabungkan judul dan narasi menjadi kolom baru


In [ ]:
df['total']=df['judul'] + ' ' + df['narasi']
df.head()

,label,judul,narasi,total
ID,,,,
71,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,Pemakaian Masker Menyebabkan Penyakit Legionna...
461,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,Instruksi Gubernur Jateng tentang penilangan ...
495,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,Foto Jim Rohn: Jokowi adalah presiden terbaik ...
550,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...","ini bukan politik, tapi kenyataan Pak Jokowi b..."
681,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,Foto Kadrun kalo lihat foto ini panas dingin K...


**2. Data Testing**

In [ ]:
df_t = pd.read_excel('/content/gdrive/My Drive/BDC SD/Data Uji BDC.xlsx')
df_t.head()

,ID,tanggal,judul,narasi,nama file gambar,label
0,238057,2020-07-13 00:00:00,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,238057.jpg,1
1,238158,2020-07-06 00:00:00,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,238158.jpg,1
2,238865,2020-04-22 00:00:00,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,238865.jpg,1
3,248298,2019-10-22 00:00:00,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,248298.jpg,1
4,255176,2020-05-01 00:00:00,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",255176.jpg,1


In [ ]:
df_t['total']=df_t['judul'] + ' ' + df_t['narasi']
df_t = df_t.drop(['tanggal', 'nama file gambar'], axis=1)
df_t.head()

,ID,judul,narasi,label,total
0,238057,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,1,Narasi Tito Karnavian Berideologi Komunis Kare...
1,238158,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,1,Anies: Seberat beratnya Pekerjaan Akan terasa ...
2,238865,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,1,Hindu di india Melemparkan Patung Buatan Merek...
3,248298,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,1,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...
4,255176,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",1,Permohonan Kelonggaran Angsuran ke OJK Untuk s...


In [ ]:
df_t = df_t.set_index('ID')

# **Preprocessing Data**

In [ ]:
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install Sastrawi
#stopwords removal
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stop = StopWordRemoverFactory().get_stop_words()

#stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()

     |████████████████████████████████| 215kB 2.6MB/s 


In [ ]:
stemmer = factory.create_stemmer()

from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
# list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
#                        'kalo', 'amp', 'biar', 'bikin', 'bilang', 
#                        'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
#                        'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
#                        'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
#                        'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
#                        '&amp', 'yah', 'lg', 'deh', 'toh', 'piye', 'prettt', 'tak', 'sblm', 'dlm', 'sekedar'])
stop_words = list_stopwords

#preprocessing data training
for index, row in df.iterrows():
    filter_sentence = ''
    sentence = row['total']
    # Cleaning dengan regular expression
    sentence = re.sub(r'[^\w\s]', '', sentence)
    # sentence = stemmer.stem(sentence)
    # Tokenization
    words = nltk.word_tokenize(sentence)
    # Menghilangkan stopwords
    words = [w for w in words if not w in stop_words]
    # Lower case
    for words in words:
        filter_sentence = filter_sentence  + ' ' + words.lower()
    
    df.loc[index, 'total'] = filter_sentence

#preprocessing data testing    
for index, row in df_t.iterrows():
    filter_sentence = ''
    sentence = row['total']
    # Cleaning dengan regular expression
    sentence = re.sub(r'[^\w\s]', '', sentence)
    # sentence = stemmer.stem(sentence)
    # Tokenization
    words = nltk.word_tokenize(sentence)
    # Menghilangkan stopwords
    words = [w for w in words if not w in stop_words]
    # Lower case
    for words in words:
        filter_sentence = filter_sentence  + ' ' + words.lower()
    
    df_t.loc[index, 'total'] = filter_sentence

In [ ]:
df.head()

,label,judul,narasi,total
ID,,,,
71,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,pemakaian masker menyebabkan penyakit legionn...
461,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,instruksi gubernur jateng penilangan yg berma...
495,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,foto jim rohn jokowi presiden terbaik dlm sej...
550,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",politik kenyataan pak jokowi berhasil memulan...
681,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,foto kadrun kalo lihat foto panas dingin kadr...


In [ ]:
#inisiasi x dan y
x_train = df['total']
y_train = df['label']

In [ ]:
df_t.head()

,judul,narasi,label,total
ID,,,,
238057,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,1,narasi tito karnavian berideologi komunis kar...
238158,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,1,anies seberat beratnya pekerjaan akan ringan ...
238865,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,1,hindu india melemparkan patung buatan mereka ...
248298,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,1,rscm praktekkan penyedotan plug venasaluran d...
255176,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",1,permohonan kelonggaran angsuran ojk untuk sek...


In [ ]:
x_test = df_t['total']
y_test = df_t['label']

# **Vectorizaton**

1. TF-IDF Vectorization

In [ ]:
# Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(max_df=1.0)

#Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

2. Count Vectorizer

In [ ]:
# Initialize a CountVectorizer
count_vectorizer = CountVectorizer()

#Fit and transform train set, transform test set
count_train = count_vectorizer.fit_transform(x_train)
count_test = count_vectorizer.transform(x_test)

# **Modelling**

1. PASSIVE-AGGRESSIVE CLASSIFIER

In [ ]:
from sklearn.metrics import f1_score
from sklearn.linear_model import PassiveAggressiveClassifier

In [ ]:
# PASSIVE-AGGRESSIVE CLASSIFIER TF-IDF   

pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)
#Predict on the test set
y_pred = pac.predict(tfidf_test)

#F1 Score
scoref1 = f1_score(y_test, y_pred)
print(f'F1 Score: {round(scoref1*100,2)}%')


F1 Score: 91.25%


In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred, labels=[0,1])
print(confusion_matrix)

[[ 35  46]
 [ 24 365]]


In [ ]:
# PASSIVE-AGGRESSIVE CLASSIFIER Count

pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(count_train,y_train)

#Predict on the test set
y_pred=pac.predict(count_test)

scoref1c=f1_score(y_test, y_pred)
print(f'F1 score: {round(scoref1c*100,2)}%')

F1 score: 88.92%


2. MULTINOMIAL NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
#Multinomial NB TF-IDF

# Create the list of alphas: alphas
alphas = np.arange(0, 1, 0.1)

# Define train_and_predict()
def train_and_predict(alpha, tfidf_train, y_train, tfidf_test, y_test):
    # Instantiate the classifier: nb_classifier
    nb_classifier = MultinomialNB(alpha=alpha)
    # Fit to the training data
    nb_classifier.fit(tfidf_train, y_train)
    # Predict the labels: pred
    pred = nb_classifier.predict(tfidf_test)
    # Compute accuracy: score
    score = metrics.f1_score(y_test, pred)
    return score

# Iterate over the alphas and print the corresponding score
for alpha in alphas:
    print('Alpha: ', alpha)
    print('Score: ', train_and_predict(alpha, tfidf_train, y_train, tfidf_test, y_test))
    print()

Alpha:  0.0
Score:  0.9009661835748793

Alpha:  0.1
Score:  0.9214285714285714

Alpha:  0.2
Score:  0.911660777385159

Alpha:  0.30000000000000004
Score:  0.9063231850117096

Alpha:  0.4
Score:  0.9067599067599067

Alpha:  0.5
Score:  0.9057043073341093

Alpha:  0.6000000000000001
Score:  0.9057043073341093

Alpha:  0.7000000000000001
Score:  0.9057043073341093

Alpha:  0.8
Score:  0.9057043073341093

Alpha:  0.9
Score:  0.9057043073341093



/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(tfidf_train, y_train)
Accuracy = logreg.score(tfidf_test, y_test)

print(Accuracy*100)

85.53191489361703


In [ ]:
hasil = pd.DataFrame()

In [ ]:
nb_classifier = MultinomialNB(alpha=0.1)
# Fit to the training data
nb_classifier.fit(tfidf_train, y_train)
# Predict the labels: pred
pred = nb_classifier.predict(tfidf_test)
# Compute accuracy: score
score = metrics.f1_score(y_test, pred)
print(score)

0.9214285714285714


In [ ]:
print(x_test.head())
display(df_t.head())
print(x_test.index)

ID
238057     narasi tito karnavian berideologi komunis kar...
238158     anies seberat beratnya pekerjaan akan ringan ...
238865     hindu india melemparkan patung buatan mereka ...
248298     rscm praktekkan penyedotan plug venasaluran d...
255176     permohonan kelonggaran angsuran ojk untuk sek...
Name: total, dtype: object


,judul,narasi,label,total
ID,,,,
238057,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,1,narasi tito karnavian berideologi komunis kar...
238158,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,1,anies seberat beratnya pekerjaan akan ringan ...
238865,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,1,hindu india melemparkan patung buatan mereka ...
248298,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,1,rscm praktekkan penyedotan plug venasaluran d...
255176,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",1,permohonan kelonggaran angsuran ojk untuk sek...


Int64Index([238057, 238158, 238865, 248298, 255176], dtype='int64', name='ID')


In [ ]:
print(pred[:5])
print(y_test[:2])

print(len(pred))
print(len(y_test))
print(len(x_test.index))

hasil_prediksi = {'ID':[i for i in x_test.index], 'prediksi':[y for y in pred]}
hasil_prediksi_df = pd.DataFrame.from_dict(hasil_prediksi)

[1 1 1 1 1]
ID
238057    1
238158    1
Name: label, dtype: int64
470
470
470


In [ ]:
print(hasil_prediksi_df.head())
print(hasil_prediksi_df.info())
hasil_prediksi_df.to_csv('hasil_pred.csv', index=False)

from google.colab import  drive
drive.mount('/drive')
hasil_prediksi_df.to_csv('/drive/My Drive/Folder tanpa nama/hasil_prediksi.csv', index=False)

       ID  prediksi
0  238057         1
1  238158         1
2  238865         1
3  248298         1
4  255176         1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   ID        470 non-null    int64
 1   prediksi  470 non-null    int64
dtypes: int64(2)
memory usage: 7.5 KB
None
Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
